In [2]:
#Importing Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import warnings


warnings.filterwarnings("ignore")

In [3]:
# locate the directory
DIRECTORY = "/home/akira/MoBI/cpcst_hopkins/"
arr = os.listdir(DIRECTORY)

In [4]:
def pre_process(subject,size):
    # The list of columns to be dropped
    drop_ls = [
    "expected_time",
    "flip_time",
    "stim_pos",
    "user_pos",
    "lambda_val",
    "change_rate_x",
    ]

    # the feature matrix in array form
    X = subject.drop(columns=drop_ls).to_numpy()
    #print("The shape of X:", X.shape)
    y_pos_dif = (subject["user_pos"].to_numpy() - subject["stim_pos"].to_numpy())
    y = y_pos_dif
    X_train, X_test, y_train, y_test = train_test_split(
    X[:size], y[:size], test_size=0.2, random_state=23)  # only 100 samples are used here
    return X_train, X_test, y_train, y_test

In [5]:
class RidgeRegression:
    def __init__(self, alpha=1.0, lr=0.01, n_iterations=1000):
        """
        Ridge Regression model with L2 regularization.

        Parameters:
        alpha : float
            Regularization strength.
        lr : float
            Learning rate for gradient descent.
        n_iterations : int
            Number of iterations for training.
        """
        self.alpha = alpha
        self.lr = lr
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def _initialize_parameters(self, n_features):
        self.weights = np.zeros(n_features)
        self.bias = 0.0

    def _compute_cost(self, X, y):
        n_samples = len(X)
        y_pred = [self.bias + sum(self.weights[j] * X[i][j] for j in range(len(self.weights))) for i in range(n_samples)]
        mse = (1 / (2 * n_samples)) * sum((y_pred[i] - y[i]) ** 2 for i in range(n_samples))
        l2_penalty = (self.alpha / (2 * n_samples)) * sum(w ** 2 for w in self.weights)
        return mse + l2_penalty

    def _compute_gradients(self, X, y):
        """
        Compute gradients of weights and bias for gradient descent, element-wise.
        """
        n_samples = len(X)
        dw = [0.0] * len(self.weights)
        db = 0.0

        # Calculate predictions and compute gradients
        for i in range(n_samples):
            y_pred_i = self.bias + sum(self.weights[j] * X[i][j] for j in range(len(self.weights)))
            error = y_pred_i - y[i]

            # Update the gradient for bias
            db += error / n_samples

            # Update the gradient for each weight separately
            for j in range(len(self.weights)):
                dw[j] += (X[i][j] * error / n_samples) + (self.alpha / n_samples) * self.weights[j]

        return dw, db

    def fit(self, X, y):
        n_features = len(X[0])
        self._initialize_parameters(n_features)

        for i in range(self.n_iterations):
            dw, db = self._compute_gradients(X, y)
            for j in range(len(self.weights)):
                self.weights[j] -= self.lr * dw[j]
            self.bias -= self.lr * db

    def predict(self, X):
        y_pred = [self.bias + sum(self.weights[j] * x[j] for j in range(len(self.weights))) for x in X]
        return y_pred
    
    def score(self, X, y):
        y_pred = self.predict(X)
        ss_res = sum((y[i] - y_pred[i]) ** 2 for i in range(len(y)))
        y_mean = sum(y) / len(y)
        ss_tot = sum((y[i] - y_mean) ** 2 for i in range(len(y)))
        r2_score = 1 - (ss_res / ss_tot)
        return r2_score

In [44]:
def cross_validate_ridge(X, y, alpha_values, k=5, lr=0.01, n_iterations=1000):
    """
    Perform K-Fold Cross-Validation to select the best alpha for Ridge Regression.

    Parameters:
    X : numpy.ndarray
        Feature matrix.
    y : numpy.ndarray
        Target vector.
    alpha_values : list of float
        List of alpha values to try.
    k : int
        Number of folds for K-Fold Cross-Validation.
    lr : float
        Learning rate for gradient descent.
    n_iterations : int
        Number of iterations for training.

    Returns:
    float
        The best alpha value based on average cross-validation score.
    """
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    best_alpha = None
    best_score = float('inf')

    for alpha in alpha_values:
        model = RidgeRegression(alpha=alpha, lr=lr, n_iterations=n_iterations)
        fold_scores = []

        for train_index, test_index in kfold.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            fold_scores.append(mse)

        average_score = np.mean(fold_scores)

        if average_score < best_score:
            best_score = average_score
            best_alpha = alpha

        print(f"Alpha: {alpha}, Average MSE: {average_score/1e177}")

    print(f"Best alpha: {best_alpha}, with Average MSE: {best_score/1e177}")
    return best_alpha

In [45]:
subject = pd.read_csv(DIRECTORY + arr[0])
size = subject.shape[0]

In [46]:
X_train, X_test, y_train, y_test = pre_process(subject,size)

In [47]:
model1 = RidgeRegression(n_iterations=10)
model1.fit(X_train, y_train)
print("Model R² Score:", model1.score(X_test, y_test)/1e180)

Model R² Score: -0.02390113533304817


In [48]:
# Example usage:
# X, y should be your features and target arrays
alpha_values = [0.01, 0.1, 1, 10, 100]  # Define a range of alpha values to try
best_alpha = cross_validate_ridge(X_train, y_train, alpha_values, k=5, lr=0.01, n_iterations=10)

# Train the final model with the best alpha
model2 = RidgeRegression(alpha=best_alpha, lr=0.01, n_iterations=10)
model2.fit(X_train, y_train)
z =  model2.score(X_test, y_test)/1e180
print("Model R² Score:", z)

Alpha: 0.01, Average MSE: 0.09603428854446681
Alpha: 0.1, Average MSE: 0.09603428854567383
Alpha: 1, Average MSE: 0.0960342885577439
Alpha: 10, Average MSE: 0.0960342886784361
Alpha: 100, Average MSE: 0.09603428988536529
Best alpha: 0.01, with Average MSE: 0.09603428854446681
Model R² Score: -0.02390113532974407


In [49]:
z

np.float64(-0.02390113532974407)

In [50]:
best_alpha

0.01